In [2]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime


In [11]:
db_user = "root"
db_password = "ortonfan007!"  
db_host = "localhost"
db_port = "3306"
db_name = "ptcl_inventory"

# SQLAlchemy connection string
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_string)


In [5]:
!pip install pymysql



  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/0c/94/e4181a1f6286f545507528c78016e00065ea913276888db2262507693ce5/PyMySQL-1.1.1-py3-none-any.whl.metadata
  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.1-py3-none-any.whl (44 kB)


In [3]:
# Load CSVs
sheet1 = pd.read_csv('../data_raw/ptcl_sheet1.csv')
sto1 = pd.read_csv('../data_raw/ptcl_sto1.csv')


In [4]:
def clean_column_names(df):
    return df.rename(columns=lambda col: col.strip().lower().replace(" ", "_").replace(".", "").replace("/", "_"))
sheet1_clean = clean_column_names(sheet1)
sto1_clean = clean_column_names(sto1)

In [5]:
# ✅ STEP 1: Clean column names
sheet1_clean = clean_column_names(sheet1)
sto1_clean = clean_column_names(sto1)

# ✅ STEP 2: Convert date columns to datetime format
sheet1_date_cols = ['posting_date', 'document_date', 'entry_date']
sto1_date_cols = ['posting_date', 'document_date', 'entry_date', 'deliv_date', 'gr_posting_date']

# For Sheet1
for col in sheet1_date_cols:
    sheet1_clean[col] = pd.to_datetime(sheet1_clean[col], format='%d/%m/%Y', errors='coerce')

# For STO1
for col in sto1_date_cols:
    if col in sto1_clean.columns:
        sto1_clean[col] = pd.to_datetime(sto1_clean[col], format='%d/%m/%Y', errors='coerce')



In [6]:
sheet1_clean = sheet1_clean.rename(columns={"amtin_loccur": "amt_in_loc_cur"})
sto1_clean = sto1_clean.rename(columns={"amtin_loccur": "amt_in_loc_cur"})


In [7]:
sheet1_clean.rename(columns={'amtin_loccur': 'amt_in_loc_cur'}, inplace=True)
sto1_clean.rename(columns={'amtin_loccur': 'amt_in_loc_cur'}, inplace=True)

In [8]:
def validate_columns(df, expected_columns, df_name="DataFrame"):
    actual = set(df.columns)
    expected = set(expected_columns)
    
    missing = expected - actual
    extra = actual - expected

    if missing:
        print(f"❌ {df_name} is missing columns: {missing}")
    if extra:
        print(f"⚠️  {df_name} has unexpected extra columns: {extra}")
    if not missing and not extra:
        print(f"✅ {df_name} columns are valid.")

# Expected schemas (no ingestion_date here)
expected_sheet1_columns = [
    'plant', 'storage_location', 'purchase_order', 'material', 'material_description',
    'reference', 'quantity', 'unit_of_entry', 'posting_date', 'amt_in_loc_cur',
    'movement_type', 'user_name', 'document_date', 'entry_date'
]

expected_sto1_columns = [
    'plant', 'storage_location', 'purchase_order', 'material', 'material_description',
    'reference', 'quantity', 'unit_of_entry', 'posting_date', 'amt_in_loc_cur',
    'movement_type', 'user_name', 'document_date', 'entry_date',
    'receiving_plant', 'deliv_date', 'gr_posting_date'
]

# ✅ Run validations before loading
validate_columns(sheet1_clean, expected_sheet1_columns, "sheet1_clean")
validate_columns(sto1_clean, expected_sto1_columns, "sto1_clean")


✅ sheet1_clean columns are valid.
✅ sto1_clean columns are valid.


In [9]:
from datetime import datetime

today = datetime.today().strftime('%Y-%m-%d')  # Use uniform format

sheet1_clean["ingestion_date"] = today
sto1_clean["ingestion_date"] = today



In [12]:
# 🚀 Load cleaned data into MySQL tables
sheet1_clean.to_sql('raw_sheet1', con=engine, if_exists='append', index=False)
sto1_clean.to_sql('raw_sto1', con=engine, if_exists='append', index=False)

print(" Lets goooo! Data successfully loaded into MySQL!")


 Lets goooo! Data successfully loaded into MySQL!
